In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
#twitter = pd.read_csv("preprocessed_data.csv")    #normal one used before
#twitter = pd.read_csv("sorted_preprocessed_data.csv")  #with lowest emotion scores
twitter = pd.read_csv("before_balancing_data.csv")

Wall time: 6.19 s


In [3]:
twitter

,_score,tweet_id,text,identification,emotion,text_stemmed,Category
0,391,0x376b20,people who post add me on snapchat must be deh...,train,anticipation,['people who post add me on snapchat must be d...,4
1,433,0x2d5350,brianklaas as we see trump is dangerous to fre...,train,sadness,['brianklaas as we see trump is dangerous to f...,8
2,376,0x1cd5b0,now issa is stalking tasha 😂😂😂,train,fear,['now issa is stalking tasha 😂😂😂 '],1
3,120,0x1d755c,riskshow thekevinallison thx for the best time...,train,joy,['riskshow thekevinallison thx for the best ti...,7
4,1021,0x2c91a8,still waiting on those supplies liscus,train,anticipation,['still waiting on those supplies liscus '],4
...,...,...,...,...,...,...,...
1451432,94,0x321566,im so happy nowonder the name of this show hap...,train,joy,['im so happy nowonder the name of this show h...,7
1451433,627,0x38959e,in every circumtance id like to be thankful to...,train,joy,['in every circumtance id like to be thankful ...,7
1451434,274,0x2cbca6,theres currently two girls walking around the ...,train,joy,['theres currently two girls walking around th...,7
1451435,840,0x24faed,ah corporate life where you can date using ju...,train,joy,['ah corporate life where you can date using ...,7


In [4]:
#Dividing the dataframe into a train and a test sections
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(twitter.text,twitter.emotion,
                                                test_size=0.2, random_state = 42)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer

#Here we get the TF_IDF for the first 100 features:
number_features = 10000

tkzr = TweetTokenizer(strip_handles = True)

tfidf_vectorizer = TfidfVectorizer(
                            lowercase=True,
                            max_features= number_features,
                            max_df=0.8,
                            min_df=2,
                            #ngram_range = (1,3),
                            stop_words = "english",
                            tokenizer = tkzr.tokenize
                            )

vectors = tfidf_vectorizer.fit_transform(x_train)
feature_names = tfidf_vectorizer.get_feature_names()

In [6]:
#dense = vectors.todense()
#denselist = dense.tolist()
#df_TFIDF = pd.DataFrame(vectors.toarray(), columns=feature_names)
#df_TFIDF

#vectors = vectors.toarray()

In [7]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
naive_classifier = MultinomialNB()
naive_classifier.fit(vectors, y_train)

MultinomialNB()

In [8]:
x_test = tfidf_vectorizer.transform(x_test)
x_test = x_test.toarray()

MemoryError: Unable to allocate 21.6 GiB for an array with shape (290288, 10000) and data type float64

In [ ]:
#Make the predicitions with the test data
predictions = naive_classifier.predict(x_test)

#Establish how many correct, incorrect predictions, and accuracy:
correct_predictions = (y_test == predictions).sum()
incorrect_predictions = y_test.size - correct_predictions
accuracy = round((naive_classifier.score(x_test,y_test))*100,2)

results_naive = correct_predictions, incorrect_predictions, accuracy

df_results = pd.DataFrame([results_naive], columns = ['Correct','Incorrect', 'Accuracy %'])
df_results

#with 1000 features best accuracy : 46.29%
#with 2000 features best accuracy : 48.35%
#with 5000 features best accuracy : 50.16%

### Predicting on new Public Testing Data

In [ ]:
twitter_test_data = pd.read_csv("twitter_test_data.csv")

In [ ]:
x_test_twitter = tfidf_vectorizer.transform(twitter_test_data['text'])
pred_result_test_data = naive_classifier.predict(x_test_twitter)

In [ ]:
pred_result_test_data

In [ ]:
upload_df = pd.DataFrame(columns = [["id","emotion"]])
upload_df["id"] = twitter_test_data["tweet_id"]
upload_df["emotion"] = pred_result_test_data
upload_df

In [ ]:
upload_df.to_csv("./uploads/Naive_Bayes10k.csv",index=False)